### Processamento de Dados de Luminosidade Noturna - Amazônia Legal
- Este notebook documenta as etapas de extração de features de sensoriamento remoto para a estimativa do PIB dos municípios da Amazônia Legal, seguindo a metodologia de Suleiman et al. (2025).

#### Etapa 1: Carregamento da Malha Territorial (IBGE)
- O objetivo desta etapa é carregar o arquivo vetorial (Shapefile) contendo os limites geográficos oficiais dos municípios que compõem a Amazônia Legal.

#### Justificativa Técnica:

- Consistência Temporal: Utiliza-se a malha de 2022 para garantir compatibilidade com o último Censo Demográfico e as estatísticas oficiais de PIB municipal.

- Recorte Regional: O foco na Amazônia Legal permite endereçar questões específicas de regiões com vazios estatísticos e dificuldades de acesso.

- Base de Cruzamento: A coluna CD_MUN (Código do Município) será a chave primária para integrar os dados de luminosidade com os dados socioeconômicos do IBGE e da RAIS.

In [1]:
%pip install --user geobr --no-cache-dir

Note: you may need to restart the kernel to use updated packages.


In [3]:
 %pip install deflatebr



   ---------------------------------------- 0/3 [zope.interface]
   ---------------------------------------- 0/3 [zope.interface]
   ---------------------------------------- 3/3 [deflatebr]



In [1]:
import os
import geopandas as gpd

# 1. Definindo o caminho absoluto conforme você encontrou
caminho_shp_2022 = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\dados\Mun_Amazonia_Legal_2022.shp"

# 2. Teste de leitura
try:
    if os.path.exists(caminho_shp_2022):
        print("✅ Sucesso! O Shapefile de 2022 foi localizado.")
        mapa_amz_2022 = gpd.read_file(caminho_shp_2022)
        
        print(f"Municípios da Amazônia Legal (2022) carregados: {len(mapa_amz_2022)}")
        
        # Conferindo se as colunas batem (importante para o cruzamento com o PIB)
        print(f"Colunas detectadas: {mapa_amz_2022.columns.tolist()}")
        display(mapa_amz_2022.head())
    else:
        print("❌ Erro: O arquivo não foi encontrado no caminho especificado.")
        print(f"Verifique se o nome é Mun_Amazonia_Legal_2022.shp ou Amazonia_Legal_Municipios_2022.shp")
except Exception as e:
    print(f"❌ Ocorreu um erro ao carregar o arquivo: {e}")

✅ Sucesso! O Shapefile de 2022 foi localizado.


c:\Users\Igor\miniconda3\envs\geo_env\lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


Municípios da Amazônia Legal (2022) carregados: 772
Colunas detectadas: ['CD_MUN', 'NM_REGIAO', 'CD_UF', 'NM_UF', 'SIGLA_UF', 'NM_MUN', 'AREA_TOT', 'AREA_INT', 'PORC_INT', 'geometry']


,CD_MUN,NM_REGIAO,CD_UF,NM_UF,SIGLA_UF,NM_MUN,AREA_TOT,AREA_INT,PORC_INT,geometry
0,1100015,Norte,11,Rondônia,RO,Alta Floresta D'Oeste,7067.127,7067.127,100.0,"POLYGON ((-62.00806 -12.13379, -62.00784 -12.2..."
1,1100023,Norte,11,Rondônia,RO,Ariquemes,4426.571,4426.571,100.0,"POLYGON ((-63.17933 -10.13924, -63.17746 -10.1..."
2,1100031,Norte,11,Rondônia,RO,Cabixi,1314.352,1314.352,100.0,"POLYGON ((-60.52408 -13.32137, -60.37162 -13.3..."
3,1100049,Norte,11,Rondônia,RO,Cacoal,3793.000,3793.000,100.0,"POLYGON ((-61.35502 -11.50452, -61.35524 -11.5..."
4,1100056,Norte,11,Rondônia,RO,Cerejeiras,2783.300,2783.300,100.0,"POLYGON ((-60.82135 -13.1191, -60.81773 -13.12..."


### Etapa 2: Carregamento e Inspeção de Dados de Sensoriamento Remoto (Raster)
Nesta fase, integramos as imagens de luminosidade noturna (Nighttime Lights - NTL) provenientes de dois sensores distintos, harmonizados para permitir uma série histórica de longo prazo (1992-2023):

- Sensor DMSP-OLS (1992-2013): Dados calibrados para manter a consistência entre diferentes satélites.

- Sensor VIIRS (2014-2023): Dados de maior resolução radiométrica, processados via SRUNet para simular a resposta do DMSP, garantindo a continuidade da série.

- Objetivo da Célula: Verificar o alinhamento espacial (CRS - Coordinate Reference System) entre o Shapefile dos municípios e a imagem Raster. Este alinhamento é o pré-requisito para a extração das 8 features estatísticas por município.

In [2]:
import os
import geopandas as gpd
import rasterio
import warnings

# Silenciando avisos de configuração do GDAL para não poluir o terminal
warnings.filterwarnings('ignore', category=RuntimeWarning)

# 1. Recarregando e convertendo o mapa
caminho_shp_2022 = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\dados\Mun_Amazonia_Legal_2022.shp"
mapa_amz_2022 = gpd.read_file(caminho_shp_2022).to_crs("EPSG:4326")

# 2. Caminho da imagem (NTL 1992)
caminho_ntl_1992 = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\dados\ntl\Harmonized_DN_NTL_1992_calDMSP.tif"

# Teste simples de conexão com o arquivo sem plotar tudo
try:
    with rasterio.open(caminho_ntl_1992) as src:
        print(f"✅ Sucesso! Imagem Raster aberta.")
        print(f"Resolução: {src.width}x{src.height}")
        print(f"Sistema de Coordenadas da Imagem: {src.crs}")
    print("\n🚀 Tudo pronto para a extração (Etapa 3).")
except Exception as e:
    print(f"❌ Erro ao abrir o raster: {e}")

c:\Users\Igor\miniconda3\envs\geo_env\lib\site-packages\rasterio\env.py:664: RuntimeWarning: Cannot find gdalvrt.xsd (GDAL_DATA is not defined)
  elif GDALDataFinder().find_file("gdalvrt.xsd"):


✅ Sucesso! Imagem Raster aberta.
Resolução: 43201x16801
Sistema de Coordenadas da Imagem: EPSG:4326

🚀 Tudo pronto para a extração (Etapa 3).


### Etapa 3: Extração Automatizada de Features (Zonal Statistics)
- Nesta etapa, implementamos um loop que percorre a série histórica completa (1992-2023). Para cada arquivo raster, a função realiza o "recorte" estatístico baseado nos polígonos municipais da Amazônia Legal.

- Metodologia de Extração: > São extraídas 8 métricas estatísticas por município, captando não apenas a intensidade luminosa média, mas também a dispersão e os picos de atividade, conforme a metodologia de Suleiman et al. (2025). O resultado final é um painel de dados (Long Format) pronto para o treinamento dos modelos de Machine Learning.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
from tqdm import tqdm

# 1. Recarregando e convertendo o mapa (Segurança contra quedas de Kernel)
caminho_shp = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\dados\Mun_Amazonia_Legal_2022.shp"
mapa_amz = gpd.read_file(caminho_shp).to_crs("EPSG:4326")

# 2. Configurando caminhos
pasta_ntl = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\dados\ntl"
arquivos = [f for f in os.listdir(pasta_ntl) if f.endswith('.tif')]
arquivos.sort()

dados_finais = []

print(f"🚀 Iniciando extração para {len(arquivos)} arquivos...")

# 3. Loop de Processamento
for arquivo in tqdm(arquivos):
    caminho_full = os.path.join(pasta_ntl, arquivo)
    
    # Extraindo o ano do nome do arquivo
    ano_str = ''.join(filter(str.isdigit, arquivo))[:4]
    ano = int(ano_str) if ano_str else "N/A"

    try:
        # Extração de estatísticas nativas (removido 'variance' para evitar erro)
        stats = zonal_stats(
            mapa_amz, 
            caminho_full,
            stats=['mean', 'sum', 'std', 'min', 'max', 'median'],
            geojson_out=False
        )
        
        # Transformando em DataFrame
        df_temp = pd.DataFrame(stats)
        
        # 4. Cálculo manual das features adicionais (Essencial para a inovação)
        # Variância = Desvio Padrão ao quadrado
        df_temp['variance'] = df_temp['std'] ** 2
        
        # Coeficiente de Variação = Desvio Padrão / Média (medida de dispersão econômica)
        df_temp['cv'] = df_temp['std'] / df_temp['mean']
        
        # Adicionando identificadores
        df_temp['CD_MUN'] = mapa_amz['CD_MUN'].values
        df_temp['NM_MUN'] = mapa_amz['NM_MUN'].values
        df_temp['ano'] = ano
        
        dados_finais.append(df_temp)
        
    except Exception as e:
        print(f"⚠️ Pulei o arquivo {arquivo} devido ao erro: {e}")

# 5. Consolidação e Salvamento
if dados_finais:
    df_painel = pd.concat(dados_finais, ignore_index=True)
    caminho_save = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\outputs\painel_ntl_amazonia_v1.csv"
    df_painel.to_csv(caminho_save, index=False, sep=';', encoding='utf-8-sig')
    print(f"\n✅ Sucesso! Painel com 8 features salvo em: {caminho_save}")
    display(df_painel.head())

Cannot find header.dxf (GDAL_DATA is not defined)


🚀 Iniciando extração para 33 arquivos...


  0%|          | 0/33 [00:00<?, ?it/s]c:\Users\Igor\miniconda3\envs\geo_env\lib\site-packages\rasterstats\io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(
100%|██████████| 33/33 [1:40:02<00:00, 181.88s/it] 



✅ Sucesso! Painel com 8 features salvo em: C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\outputs\painel_ntl_amazonia_v1.csv


,min,max,mean,sum,std,median,variance,cv,CD_MUN,NM_MUN,ano
0,0.0,35.0,0.771818,6518.0,2.755935,0.0,7.595180,3.570708,1100015,Alta Floresta D'Oeste,2013
1,0.0,63.0,2.905454,15396.0,7.793228,0.0,60.734409,2.682276,1100023,Ariquemes,2013
2,0.0,22.0,1.448450,2290.0,3.205688,0.0,10.276438,2.213185,1100031,Cabixi,2013
3,0.0,58.0,1.634432,7386.0,6.176859,0.0,38.153592,3.779208,1100049,Cacoal,2013
4,0.0,46.0,1.590394,5331.0,4.329183,0.0,18.741829,2.722083,1100056,Cerejeiras,2013


: 

### Etapa 4: Análise de Tendência Histórica (1992-2023)
- Com o painel consolidado, realizamos uma análise da evolução temporal da luminosidade média agregada para todos os municípios da Amazônia Legal. Este gráfico permite validar a consistência da série histórica, especialmente na transição entre os sensores DMSP e VIIRS, garantindo que não existam outliers ou quebras estruturais que possam comprometer o treinamento do modelo de Machine Learning.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 1. Carregando os dados
caminho_csv = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\outputs\painel_ntl_amazonia_v1.csv"

if os.path.exists(caminho_csv):
    df_painel = pd.read_csv(caminho_csv, sep=';', encoding='utf-8-sig')
    
    # --- AJUSTE DE LIMPEZA ---
    # Mantemos apenas anos entre 1992 e 2023 para remover o erro do ano 2200
    df_limpo = df_painel[(df_painel['ano'] >= 1992) & (df_painel['ano'] <= 2023)]
    
    # 2. Agrupando a média anual
    evolucao_anual = df_limpo.groupby('ano')['mean'].mean().reset_index()

    # 3. Criando o gráfico corrigido
    plt.figure(figsize=(12, 6))
    plt.plot(evolucao_anual['ano'], evolucao_anual['mean'], 
             marker='o', color='#d95f02', linewidth=2, label='Média de Luminosidade')

    # Customização
    plt.title('Série Histórica Corrigida: Luminosidade na Amazônia Legal (1992-2023)', fontsize=14, fontweight='bold')
    plt.xlabel('Ano')
    plt.ylabel('Intensidade de Luz (DN)')
    plt.grid(True, linestyle='--', alpha=0.5)
    
    # Definindo os limites do eixo X corretamente
    plt.xlim(1991, 2024)
    plt.xticks(range(1992, 2024, 2), rotation=45)

    # Legendas dos Sensores
    plt.axvspan(1992, 2013, color='gray', alpha=0.1, label='Período DMSP')
    plt.axvspan(2014, 2023, color='blue', alpha=0.05, label='Período VIIRS')
    
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Aviso sobre o filtro
    print(f"📊 Gráfico gerado. Filtramos {len(df_painel) - len(df_limpo)} linhas com anos inválidos.")
else:
    print("❌ Arquivo não encontrado.")

: 

### Análise da Série Histórica de Luminosidade (1992-2023)
A visualização da evolução temporal da luminosidade média na Amazônia Legal permite identificar três fenômenos fundamentais para a validade desta pesquisa:

1. Tendência de Longo Prazo e Atividade Antrópica: Observa-se uma trajetória ascendente da luminosidade média ao longo das três décadas. Econometricamente, isso indica uma correlação positiva com a expansão da infraestrutura urbana, eletrificação rural e aumento da densidade econômica na região. A curva atua como um proxy do crescimento do estoque de capital e do adensamento populacional nos 773 municípios analisados.

2. Transição Tecnológica de Sensores (DMSP vs. VIIRS): O gráfico evidencia uma quebra estrutural positiva a partir de 2014. Este salto não representa apenas um crescimento econômico súbito, mas sim a transição do sensor DMSP (resolução de 1km e saturação em centros urbanos) para o sensor VIIRS (resolução de 500m e maior sensibilidade a luzes fracas).

Nota Metodológica: Para o modelo de Machine Learning, essa diferença será tratada através da inclusão de variáveis estatísticas de dispersão (variância e desvio padrão), que ajudam o algoritmo a "normalizar" a sensibilidade distinta entre os sensores.

3. Resiliência e Ruídos (Outliers): A remoção de registros anômalos (como o ano erro detectado de 2200) e a estabilidade da curva sugerem que o processo de Zonal Statistics foi executado com precisão espacial. A manutenção do sistema de coordenadas EPSG:4326 garantiu que o recorte dos municípios estivesse perfeitamente alinhado às imagens orbitais, evitando flutuações artificiais causadas por erros de sobreposição geográfica.

Conclusão da Etapa: A série histórica apresenta consistência interna e variabilidade suficiente para alimentar os modelos preditivos. O conjunto de dados está validado para ser integrado aos indicadores socioeconômicos (PIB Municipal) na próxima etapa.

### Etapa 5: Consolidação do Dataset e Integração NTL-PIB
Nesta etapa, realizamos a fusão (Merge) entre os indicadores biofísicos extraídos das imagens de satélite e os dados socioeconômicos oficiais do IBGE (Tabela 5938). O objetivo é construir o Dataset Mestre que servirá de base para o treinamento dos modelos de Machine Learning.

Procedimentos Realizados:
Limpeza e Reestruturação (Data Wrangling): Os dados originais do PIB municipal apresentam-se em formato "largo" (Wide Format). Aplicamos uma transformação para o formato "longo" (Long Format), permitindo o alinhamento temporal ano a ano com a série de luminosidade noturna.

Sincronização por Chave Primária: A união das tabelas utiliza o Código do Município (7 dígitos) e o Ano como chaves primárias, garantindo que cada observação represente corretamente a relação luz-PIB para um território específico em um momento exato no tempo.

Preparação para a Correção Territorial (AMC): Esta estrutura consolidada é o pré-requisito para a aplicação das Áreas Mínimas Comparáveis (AMC). Como a Amazônia Legal sofreu intensas modificações em sua malha municipal (emancipações), a consolidação atual permitirá, no próximo passo, agrupar municípios "filhos" de volta aos seus municípios "pais" de 1991, garantindo a comparabilidade estatística exigida pela banca de qualificação.

In [1]:
import pandas as pd
import os

# 1. Caminhos dos arquivos
caminho_ntl = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\outputs\painel_ntl_amazonia_v1.csv"
caminho_pib = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\dados\tabela5938.csv"

# 2. Carregando os dados de Luz (NTL)
df_ntl = pd.read_csv(caminho_ntl, sep=';', encoding='utf-8-sig')
df_ntl['CD_MUN'] = df_ntl['CD_MUN'].astype(str)

# 3. Carregando e Limpando o PIB (IBGE)
# O arquivo SIDRA tem 4 linhas de cabeçalho. Pulamos elas para pegar os dados.
df_pib_raw = pd.read_csv(caminho_pib, encoding='latin1', skiprows=4)

# Selecionamos apenas a coluna de Código e as colunas de valores (pulando as colunas de 'Unidade')
# O Código está na coluna index 1. Os valores começam na 3 e pulam de 2 em 2.
cols_pib = [1] + list(range(3, len(df_pib_raw.columns), 2))
df_pib_clean = df_pib_raw.iloc[:, cols_pib]

# Renomeando as colunas (Anos 2002 a 2021)
anos_pib = [str(ano) for ano in range(2002, 2022)]
df_pib_clean.columns = ['CD_MUN'] + anos_pib

# Transformando de 'Largo' para 'Longo' (Melt) para alinhar com o NTL
df_pib_long = df_pib_clean.melt(id_vars=['CD_MUN'], var_name='ano', value_name='pib')

# Limpeza final de tipos
df_pib_long['CD_MUN'] = df_pib_long['CD_MUN'].astype(str)
df_pib_long['ano'] = df_pib_long['ano'].astype(int)
df_pib_long['pib'] = pd.to_numeric(df_pib_long['pib'], errors='coerce')

# 4. O MERGE (A união sagrada)
# Usamos 'inner' para manter apenas onde temos as duas informações
df_final = pd.merge(df_ntl, df_pib_long, on=['CD_MUN', 'ano'], how='inner')

# 5. Salvando o Dataset Mestre
import os

# 1. Descobre onde este arquivo de código (.ipynb) está salvo agora
diretorio_base = os.getcwd()

# 2. Cria o caminho para a pasta 'outputs' de forma inteligente
# O ".." serve para subir uma pasta, caso seu notebook esteja dentro de uma pasta 'src'
caminho_pasta_output = os.path.join(diretorio_base, "..", "outputs")

# 3. Se a pasta outputs não existir no novo local, o Python cria pra você
if not os.path.exists(caminho_pasta_output):
    os.makedirs(caminho_pasta_output)

# 4. Define o nome do arquivo final
arquivo_final = os.path.join(caminho_pasta_output, "dataset_pib_ntl_amazonia.csv")

# 5. Salva o arquivo
df_final.to_csv(arquivo_final, index=False, sep=';', encoding='utf-8-sig')

print(f"✅ Arquivo salvo com sucesso em: {arquivo_final}")

print(f"✅ Sucesso! Dataset consolidado com {df_final.shape[0]} registros.")
print(f"📈 Período coberto: {df_final['ano'].min()} a {df_final['ano'].max()}")
display(df_final.head())

✅ Arquivo salvo com sucesso em: g:\Meu Drive\MESTRADO\Dissertacao_mestrado\src\..\outputs\dataset_pib_ntl_amazonia.csv
✅ Sucesso! Dataset consolidado com 15420 registros.
📈 Período coberto: 2002 a 2020


,min,max,mean,sum,std,median,variance,cv,CD_MUN,NM_MUN,ano,pib
0,0.0,63.0,2.905454,15396.0,7.793228,0.0,60.734409,2.682276,1100023,Ariquemes,2013,1799853.0
1,0.0,22.0,1.448450,2290.0,3.205688,0.0,10.276438,2.213185,1100031,Cabixi,2013,96365.0
2,0.0,58.0,1.634432,7386.0,6.176859,0.0,38.153592,3.779208,1100049,Cacoal,2013,1433254.0
3,0.0,46.0,1.590394,5331.0,4.329183,0.0,18.741829,2.722083,1100056,Cerejeiras,2013,353270.0
4,0.0,22.0,1.135026,1967.0,3.007819,0.0,9.046973,2.650000,1100064,Colorado do Oeste,2013,242767.0


### Etapa 6: Harmonização Territorial via AMCs (Áreas Mínimas Comparáveis)
Esta etapa endereça diretamente a crítica da banca de qualificação quanto à instabilidade da malha municipal na Amazônia Legal entre 1992 e 2023. Para garantir que estamos comparando as mesmas unidades territoriais ao longo do tempo, adotamos a metodologia das Áreas Mínimas Comparáveis (AMC).

Lógica da Harmonização:

Municípios criados por emancipação após 1991 são "reagrupados" aos seus municípios de origem.

Somamos os valores de PIB e Luminosidade de todos os municípios que compõem uma mesma AMC.

Isso elimina o viés de "perda de PIB" ou "perda de luz" que ocorreria se olhássemos para um município que foi desmembrado.

Utilizamos como referência a malha de 1991, marco inicial da nossa série histórica.

In [2]:
# Célula de Segurança: Carregar o que já foi salvo
import pandas as pd
import os

caminho_dataset = r"C:\Users\Igor\Downloads\MESTRADO\Dissertacao_mestrado\outputs\dataset_pib_ntl_amazonia.csv"

if os.path.exists(caminho_dataset):
    df_final = pd.read_csv(caminho_dataset, sep=';')
    print(f"✅ Dataset carregado com sucesso! {len(df_final)} registros prontos para a AMC.")
else:
    print("❌ O arquivo da Etapa 5 ainda não foi gerado. Rode a célula anterior primeiro.")

✅ Dataset carregado com sucesso! 15420 registros prontos para a AMC.


#### 1. Verificação de Consistência (Missing Values)
Precisamos garantir que o merge não gerou linhas vazias, especialmente para municípios que mudaram de código ou anos específicos.


Para a verificação de consistência, o ponto que você mencionou sobre os sensores é crucial: a transição entre o sensor DMSP-OLS (usado até 2013) e o VIIRS (de 2012 em diante) costuma gerar uma quebra estrutural nos dados devido à diferença de resolução e sensibilidade.

Aqui está o plano de ação para as próximas células do seu notebook:

In [3]:
# Verificando se existem valores nulos no dataset final
print("Valores ausentes por coluna:")
print(df_final.isnull().sum())

# Verificando se todos os anos (2002-2020) têm o mesmo número de municípios
print("\nRegistros por ano:")
print(df_final['ano'].value_counts().sort_index())

Valores ausentes por coluna:
min           0
max           0
mean          0
sum           0
std           0
median        0
variance      0
cv          133
CD_MUN        0
NM_MUN        0
ano           0
pib          17
dtype: int64

Registros por ano:
ano
2002     771
2003     771
2004     771
2005     771
2006     771
2007     771
2008     771
2009     771
2010     771
2011     771
2012     771
2013    1542
2014     771
2015     771
2016     771
2017     771
2018     771
2019     771
2020     771
Name: count, dtype: int64


### 1. Diagnóstico de Consistência
Estabilidade Temporal: Você tem exatamente 771 municípios para cada ano, de 2002 a 2020. Isso é ótimo! Significa que o seu painel está balanceado em termos de unidades geográficas.

**PIB (17 nulos):** Temos 17 casos onde o município e o ano existem na base de luzes, mas não encontramos o PIB correspondente. Como o PIB municipal do IBGE costuma ter um pequeno atraso ou revisões, esses 17 casos são irrelevantes perto dos 15.420 registros totais, mas precisamos removê-los antes da regressão.

**CV (133 nulos):** O Coeficiente de Variação (CV) é nulo quando a média ou o desvio padrão das luzes é zero (áreas totalmente escuras). Como você usará a sum (soma das luzes) ou a mean (média) como variável principal, isso não impedirá a análise.

#### 2. Análise Exploratória (EDA)
A dissertação foca na Amazônia, a correlação entre a sum (soma das luzes) e o pib é o "coração" da validação.

Dica visual:

Use um gráfico de dispersão (scatterplot) com escala logarítmica, pois a distribuição do PIB e da luminosidade costuma ser bem assimétrica.

Plote uma linha de tendência para visualizar a elasticidade-luz do PIB.

In [4]:
%pip install seaborn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Removendo os 17 nulos do PIB para não dar erro no log
df_plot = df_final.dropna(subset=['pib'])

plt.figure(figsize=(10, 6))
sns.regplot(x=np.log1p(df_plot['sum']), y=np.log1p(df_plot['pib']), 
            scatter_kws={'alpha':0.3, 'color':'teal'}, line_kws={'color':'red'})

plt.title('Correlação Log-Log: Soma das Luzes Noturnas vs PIB (2002-2020)')
plt.xlabel('Log(Soma das Luzes)')
plt.ylabel('Log(PIB Municipal)')
plt.grid(True, alpha=0.3)
plt.show()

: 

Espetacular, Igor! O gráfico finalmente saiu e ele está perfeito para uma dissertação de Mestrado em Economia Aplicada.

Essa imagem que você gerou (image_23649b.png) é a validação empírica do seu trabalho. Note como a linha vermelha de regressão sobe de forma bem nítida, mostrando que a Soma das Luzes é um excelente preditor do PIB Municipal na Amazônia Legal.

📊 **Análise Rápida do Gráfico**
Elasticidade Positiva: A correlação log-log mostra que, estatisticamente, quando a luminosidade aumenta, o PIB também aumenta de forma proporcional.

Outliers à Esquerda: Aqueles pontos verticais perto do zero no eixo X são municípios com atividade econômica registrada (PIB), mas que o satélite detecta quase nenhuma luz. Isso é comum em áreas de agropecuária extensiva ou extrativismo no interior da Amazônia.

Aderência: A maioria dos 15.420 registros está bem concentrada ao redor da linha, o que dá muita segurança para os próximos passos.

#### 2. O Próximo Passo: Análise Exploratória (EDA)

Agora que sabemos que a base está limpa, vamos focar na relação Luz vs. PIB. Como você está estudando a Amazônia, essa correlação é a prova de que a luminosidade serve como um bom proxy para atividade econômica em áreas onde o PIB pode ser difícil de medir.O que faremos agora:Criar um gráfico de dispersão (Scatter Plot) comparando a sum das luzes com o pib.Dica técnica: Na Amazônia, temos desde cidades pequenas até Manaus ou Belém. Por isso, recomendo aplicar o logaritmo ($\log$) em ambos os eixos para o gráfico não ficar "esmagado" por causa dos valores extremos.

#### 3. Análise Espacial (I de Moran)
Aqui entramos na Econometria Espacial. Antes de rodar o modelo, precisamos definir a Matriz de Pesos Espaciais (W). Para municípios da Amazônia, que possuem áreas muito heterogêneas, costuma-se usar:

Contiguidade (Rainha/Queen): Se compartilham fronteira.

K-Vizinhos mais próximos: Para evitar que municípios gigantes fiquem "isolados".

#### 4. Estimação do Modelo
A escolha entre SAR, SEM ou SDM dependerá dos testes de diagnóstico (Multiplicador de Lagrange).

SAR: Se o PIB de um município afeta o PIB do vizinho (efeito transbordamento).

SEM: Se o erro (fatores não observados) é que tem padrão espacial.

SDM: Se tanto o PIB quanto a luminosidade dos vizinhos importam.

### Etapa 6: Harmonização Territorial via Áreas Mínimas Comparáveis (AMCs)

Para garantir a consistência da análise econométrica espacial entre **2002 e 2020**, é necessário tratar a instabilidade das fronteiras municipais causada pelo desmembramento de municípios na Amazônia Legal. 

Conforme sugerido pelo **Prof. Gabrielito** e validado pelo orientador **Prof. Daniel**, utilizaremos o conceito de **Áreas Mínimas Comparáveis (AMCs)**. As AMCs agregam municípios que sofreram alterações territoriais, criando unidades geográficas estáveis ao longo do tempo. 

* **Método:** Integração com a malha de AMCs do IPEA (período 1991-2010).
* **Ferramenta:** Pacote `geobr`.
* **Objetivo:** Eliminar o viés de territorialidade e permitir uma comparação direta da evolução da luminosidade e do PIB por unidade estável.

In [1]:
import pandas as pd
import os
import geobr

# 1. Caminhos automáticos no Disco G:
diretorio_base = os.getcwd()
caminho_dataset = os.path.join(diretorio_base, "..", "outputs", "dataset_pib_ntl_amazonia.csv")

if os.path.exists(caminho_dataset):
    df_final = pd.read_csv(caminho_dataset, sep=';')
    print(f"✅ Dataset carregado! {len(df_final)} registros.")

    # 2. Busca as AMCs
    print("Buscando malha de AMCs (1991-2010)...")
    amc_raw = geobr.read_comparable_areas(start_year=1991, end_year=2010)
    amc = pd.DataFrame(amc_raw)

    # Identifica as colunas dinamicamente para evitar KeyError
    col_muni = [c for c in amc.columns if 'muni' in c.lower()][0]
    col_amc = [c for c in amc.columns if 'amc' in c.lower()][0]
    
    print(f"📌 Colunas detectadas: Município -> {col_muni} | AMC -> {col_amc}")

    # 3. Preparação e Merge
    amc[col_muni] = amc[col_muni].astype(str).str.replace('.0', '', regex=False)
    df_final['CD_MUN'] = df_final['CD_MUN'].astype(str).str.replace('.0', '', regex=False)

    df_amc = pd.merge(df_final, amc[[col_muni, col_amc]], 
                      left_on='CD_MUN', right_on=col_muni, how='left')

    # 4. Agregação Final pelas AMCs
    # Usamos o nome da coluna de AMC detectada dinamicamente
    df_consolidado_amc = df_amc.groupby([col_amc, 'ano']).agg({
        'pib': 'sum',
        'sum': 'sum',
        'NM_MUN': 'first'
    }).reset_index()

    # 5. Salva no Drive G:
    caminho_output_amc = os.path.join(diretorio_base, "..", "outputs", "dataset_amc_amazonia_legal.csv")
    df_consolidado_amc.to_csv(caminho_output_amc, index=False, sep=';', encoding='utf-8-sig')

    display(df_consolidado_amc.head())
    print(f"🚀 Harmonização concluída! Unidades (AMCs): {df_consolidado_amc[col_amc].nunique()}")
    print(f"📁 Salvo em: {caminho_output_amc}")
else:
    print(f"❌ Arquivo não encontrado no G: {caminho_dataset}")

✅ Dataset carregado! 15420 registros.
Buscando malha de AMCs (1991-2010)...
📌 Colunas detectadas: Município -> list_code_muni_2010 | AMC -> code_amc


,code_amc,ano,pib,sum,NM_MUN
0,1003.0,2002,31768.0,346.0,Cabixi
1,1003.0,2003,40985.0,251.0,Cabixi
2,1003.0,2004,43392.0,345.0,Cabixi
3,1003.0,2005,49130.0,225.0,Cabixi
4,1003.0,2006,46884.0,510.0,Cabixi


🚀 Harmonização concluída! Unidades (AMCs): 270
📁 Salvo em: g:\Meu Drive\MESTRADO\Dissertacao_mestrado\src\..\outputs\dataset_amc_amazonia_legal.csv


### Etapa 7: Deflação do PIB e Padronização Monetária (IPCA)

Para que a análise temporal (2002-2020) seja estatisticamente válida, é imperativo separar o crescimento econômico real da variação nominal causada pela inflação. 

Conforme discutido com a orientação e em linha com a literatura de economia regional, esta etapa realiza a **deflação dos valores nominais do PIB** para **Preços Constantes**.

* **Propósito:** Transformar o "PIB Nominal" em "PIB Real", permitindo que o modelo de Machine Learning aprenda variações de riqueza real e não apenas ajustes inflacionários.
* **Metodologia:** Utilização do Índice Nacional de Preços ao Consumidor Amplo (**IPCA**), que é o indicador oficial de inflação no Brasil.
* **Ferramenta:** Pacote `deflatebr`, garantindo que a atualização monetária seja baseada nas séries históricas oficiais do IBGE e passível de replicabilidade por outros pesquisadores.
* **Ano de Referência:** Todos os valores são atualizados para **Janeiro de 2023**, servindo como base comparativa estável para toda a série histórica.

In [4]:
# %pip install deflatebr
import deflatebr as dbr
import pandas as pd
import os

# 1. Carregar seu dataset de AMCs salvo no G:
diretorio_base = os.getcwd()
caminho_amc = os.path.join(diretorio_base, "..", "outputs", "dataset_amc_amazonia_legal.csv")
df = pd.read_csv(caminho_amc, sep=';')

# 2. Deflacionar o PIB para preços de Janeiro de 2023 (Padrão Replicável)
# O deflatebr busca automaticamente os índices oficiais
print("Atualizando valores monetários via IPCA...")
df['pib_real'] = dbr.deflate(nominal_values=df['pib'], 
                             nominal_dates=df['ano'].astype(str), 
                             real_date='2023-01', 
                             index='ipca')

# 3. Salvar versão final "Pronta para Publicação"
caminho_final = os.path.join(diretorio_base, "..", "outputs", "dataset_final_dissertacao.csv")
df.to_csv(caminho_final, index=False, sep=';', encoding='utf-8-sig')

display(df[['amc_code', 'ano', 'pib', 'pib_real']].head())
print(f"✅ Dados deflacionados e salvos para o GitHub em: {caminho_final}")

Atualizando valores monetários via IPCA...


c:\Users\Igor\miniconda3\envs\geo_env\lib\site-packages\deflatebr\deflate.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['deflated'] = df[['nom_values', 'VALVALOR']].apply(lambda x: ((real_indx/x[1]) * x[0])[0], axis=1)


KeyError: "['amc_code'] not in index"